<a href="https://colab.research.google.com/github/vyolete/SystemRecomendations/blob/main/clean_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de limpieza de datos para un sistema de recomendación basado en un modelo de IA Kmeans

Este JupyterNotebook realiza la limpieza de los datos con el que se ha implementado el modelo.

Los datos de Obtienen de la página Kaggle

# Limpieza de los dataset para el modelo

Los data sets se obtiene directamente de: https://www.kaggle.com/datasets/saurabhbagchi/books-dataset?resource=download




# Librerías a utilizar

In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import Normalizer,LabelEncoder

sns.set(style="whitegrid")

# Conexión con drive google

In [ ]:
# este codigo Solo en Colab
try:
    import google.colab
    !pip install -q gdown
except:
    pass


# Descarga de Archivos

In [ ]:
# Descargar archivos CSV desde Google Drive
id_books = '19pQK9PIXLy2xY8-T1R_LLpEq6dYpNFQK'
id_ratings = '1HgfqNVmFjDX2rP75M9dDP_f9RmLZ0Mg_'
id_users = '1epRTV_2tuWv2sU1475cLi8xbgKRjY5aQ'

gdown.download(f'https://drive.google.com/uc?id={id_books}', 'books.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={id_ratings}', 'ratings.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={id_users}', 'users.csv', quiet=False)



Downloading...
From: https://drive.google.com/uc?id=19pQK9PIXLy2xY8-T1R_LLpEq6dYpNFQK
To: /content/books.csv
100%|██████████| 77.8M/77.8M [00:01<00:00, 64.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HgfqNVmFjDX2rP75M9dDP_f9RmLZ0Mg_
To: /content/ratings.csv
100%|██████████| 30.7M/30.7M [00:00<00:00, 64.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1epRTV_2tuWv2sU1475cLi8xbgKRjY5aQ
To: /content/users.csv
100%|██████████| 12.3M/12.3M [00:00<00:00, 110MB/s] 


'users.csv'

# Cargar Archivos

In [ ]:
# Cargar datasets originales
books_df = pd.read_csv('books.csv', sep=';', encoding='latin-1', on_bad_lines='warn', dtype=str)
ratings_df = pd.read_csv('ratings.csv', sep=';', encoding='latin-1', on_bad_lines='warn')
users_df = pd.read_csv('users.csv', sep=';', encoding='latin-1', on_bad_lines='warn', dtype=str)

<ipython-input-68-2265075249>:2: ParserWarning:

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9


<ipython-input-68-2265075249>:2: ParserWarning:

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9


<ipython-input-68-2265075249>:2: ParserWarning:

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9


<ipython-input-68-2265075249>:2: ParserWarning:

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
S

# Exploración Inicial de los datos

Revisamos por dataset lo siguiente:

* Cantidad de filas y columnas
* Lista de columnas
* Información base, como tipos de columna (objet, int)Etc.., conteo de valores null por columna.
* Valores duplicados





In [ ]:
def explore_dataset(name, df, key_duplicate=None):
    print(f"\n Dataset {name.upper()} ")
    print("\n" + "*" * 60, "\n")
    print(f"* Filas y columnas: {df.shape}")
    print(f"* Columnas: {list(df.columns)}")
    print("\nℹ* Información general:")
    print(df.info())
    print("\n* Nulos por columna:")
    print(df.isnull().sum())
    if key_duplicate:
        print(f"\n* Duplicados por {key_duplicate}: {df.duplicated(subset=key_duplicate).sum()}")
    else:
        print(f"\n* Duplicados totales: {df.duplicated().sum()}")
    print("\n" + "*" * 60)

In [ ]:
explore_dataset("Libros", books_df, key_duplicate='ISBN')


 Dataset LIBROS 

************************************************************ 

* Filas y columnas: (266739, 5)
* Columnas: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']

ℹ* Información general:
<class 'pandas.core.frame.DataFrame'>
Index: 266739 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 266739 non-null  object
 1   Book-Title           266739 non-null  object
 2   Book-Author          266737 non-null  object
 3   Year-Of-Publication  266739 non-null  int64 
 4   Publisher            266737 non-null  object
dtypes: int64(1), object(4)
memory usage: 12.2+ MB
None

* Nulos por columna:
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
dtype: int64

* Duplicados por ISBN: 0

************************************************************


In [ ]:
explore_dataset("Valoraciones", ratings_df)


 Dataset VALORACIONES 

************************************************************ 

* Filas y columnas: (1149780, 3)
* Columnas: ['User-ID', 'ISBN', 'Book-Rating']

ℹ* Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None

* Nulos por columna:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

* Duplicados totales: 0

************************************************************


In [ ]:
explore_dataset("Usuarios", users_df, key_duplicate='User-ID')


 Dataset USUARIOS 

************************************************************ 

* Filas y columnas: (278858, 3)
* Columnas: ['User-ID', 'Location', 'Age']

ℹ* Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   User-ID   278858 non-null  object
 1   Location  278858 non-null  object
 2   Age       168096 non-null  object
dtypes: object(3)
memory usage: 6.4+ MB
None

* Nulos por columna:
User-ID          0
Location         0
Age         110762
dtype: int64

* Duplicados por User-ID: 0

************************************************************


## Visualización de valores nulos

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Libros", "Valoraciones", "Usuarios"))

# Libros
books_null = books_df.isnull().mean()
books_null = books_null[books_null > 0].sort_values()
fig.add_trace(go.Bar(x=books_null.values, y=books_null.index, orientation='h', name="Libros", marker_color='tomato'), row=1, col=1)

# Valoraciones (sin nulos)
ratings_null = ratings_df.isnull().mean()
ratings_null = ratings_null[ratings_null > 0].sort_values()
fig.add_trace(go.Bar(x=ratings_null.values, y=ratings_null.index, orientation='h', name="Valoraciones", marker_color='seagreen'), row=1, col=2)

# Usuarios
users_null = users_df.isnull().mean()
users_null = users_null[users_null > 0].sort_values()
fig.add_trace(go.Bar(x=users_null.values, y=users_null.index, orientation='h', name="Usuarios", marker_color='steelblue'), row=1, col=3)

fig.update_layout(height=400, width=1000, title_text="Proporción de Nulos por Dataset", showlegend=False)
fig.show()

## Administración de valores nulos

## Buscar valos nulos por data set

Dados los datos, observamos que, encontramos valores Null en el data set de Libros y en el de usuarios, con la columna edad.

In [ ]:
def get_index(object, value):
    posicion_list = []
    result = object.isin([value])
    columns = list(result.any()[result.any() == True].index)
    for col in columns:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            posicion_list.append((row, col))
    return posicion_list

In [ ]:
# Buscar posiciones en libros de NaN antes de limpiar
posicions_nan = get_index(books_df, np.nan)
print('Posiciones de NaN en books_df:')
for pos in posicions_nan:
    print(pos)

Posiciones de NaN en books_df:
(118033, 'Book-Author')
(187689, 'Book-Author')
(128890, 'Publisher')
(129037, 'Publisher')


In [ ]:
# Buscar posiciones en valoraciones de NaN antes de limpiar
posicions_nan = get_index(ratings_df, np.nan)
print('Posiciones de NaN en ratings_df:')
for pos in posicions_nan:
    print(pos)

Posiciones de NaN en ratings_df:


In [ ]:
# Buscar posiciones en usuarios de NaN antes de limpiar
posicions_nan = get_index(users_df, np.nan)
print('Posiciones de NaN en users_df:')
for pos in posicions_nan:
    print(pos)

# Limpieza de datos

## Limpieza data set users

In [ ]:
drop_index = users_df[users_df['Location'].str.split(",").str.len() != 3].index

In [ ]:
users_df = users_df.drop(index = drop_index)

In [ ]:
users_df[['City','State','Country']] = users_df['Location'].str.strip().str.split(",",expand = True,n=2)

users_df['City'] = users_df['City'].str.strip()
users_df['State'] = users_df['State'].str.strip()
users_df['Country'] = users_df['Country'].str.strip()

users_df = users_df.drop('Location',axis = 1)

In [ ]:
users_df.head()

,User-ID,Age,City,State,Country
0,1,NaN,nyc,new york,usa
1,2,18,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom


### Distribución edades df completo

In [ ]:
users_df['Age'] = pd.to_numeric(users_df['Age'], errors='coerce').astype('float')
ages_graph_params = users_df.groupby('Age').size().sort_index()
fig = go.Figure(
    data=[
        go.Bar(
            x=ages_graph_params.index,
            y=ages_graph_params.values
        )
    ]
)
fig.update_layout(
    title='Distribución por edad',
    xaxis_title='Edad',
    yaxis_title='Frecuencia'
)
fig.show()

In [ ]:
users_to_omit = ~users_df['Age'].between(5,100) & ~users_df['Age'].isnull()
users_df.loc[users_to_omit] = np.nan
users_df['Age'].sort_values().unique()

array([  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,
        16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
        27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,
        38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,
        49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
        60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,
        71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,
        82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,
        93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,  nan])

### Asignamos promedio de edades a los usuarios con edad nula

In [ ]:
null_age = users_df['Age'].isnull()
users_df.loc[null_age,'Age'] = users_df['Age'].mean()

### Distribución edades sin nulos

In [ ]:
users_df['Age'] = pd.to_numeric(users_df['Age'], errors='coerce').astype('float')
ages_graph_params = users_df.groupby('Age').size().sort_index()
fig = go.Figure(
    data=[
        go.Bar(
            x=ages_graph_params.index,
            y=ages_graph_params.values
        )
    ]
)
fig.update_layout(
    title='Distribución por edad',
    xaxis_title='Edad',
    yaxis_title='Frecuencia'
)
fig.show()

In [ ]:
#Converitr a entero la edad para evitar datos decimales
users_df['Age'] = users_df['Age'].astype('int')
users_df['Age'].sort_values().unique()

array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100])

In [ ]:
countries = (
    ('AF', 'AFGHANISTAN'),
    ('AL', 'ALBANIA'),
    ('DZ', 'ALGERIA'),
    ('AS', 'AMERICAN SAMOA'),
    ('AD', 'ANDORRA'),
    ('AO', 'ANGOLA'),
    ('AI', 'ANGUILLA'),
    ('AQ', 'ANTARCTICA'),
    ('AG', 'ANTIGUA AND BARBUDA'),
    ('AR', 'ARGENTINA'),
    ('AM', 'ARMENIA'),
    ('AW', 'ARUBA'),
    ('AU', 'AUSTRALIA'),
    ('AT', 'AUSTRIA'),
    ('AZ', 'AZERBAIJAN'),
    ('BS', 'BAHAMAS'),
    ('BH', 'BAHRAIN'),
    ('BD', 'BANGLADESH'),
    ('BB', 'BARBADOS'),
    ('BY', 'BELARUS'),
    ('BE', 'BELGIUM'),
    ('BZ', 'BELIZE'),
    ('BJ', 'BENIN'),
    ('BM', 'BERMUDA'),
    ('BT', 'BHUTAN'),
    ('BO', 'BOLIVIA'),
    ('BA', 'BOSNIA AND HERZEGOVINA'),
    ('BW', 'BOTSWANA'),
    ('BV', 'BOUVET ISLAND'),
    ('BR', 'BRAZIL'),
    ('IO', 'BRITISH INDIAN OCEAN TERRITORY'),
    ('BN', 'BRUNEI DARUSSALAM'),
    ('BG', 'BULGARIA'),
    ('BF', 'BURKINA FASO'),
    ('BI', 'BURUNDI'),
    ('KH', 'CAMBODIA'),
    ('CM', 'CAMEROON'),
    ('CA', 'CANADA'),
    ('CV', 'CAPE VERDE'),
    ('KY', 'CAYMAN ISLANDS'),
    ('CF', 'CENTRAL AFRICAN REPUBLIC'),
    ('TD', 'CHAD'),
    ('CL', 'CHILE'),
    ('CN', 'CHINA'),
    ('CX', 'CHRISTMAS ISLAND'),
    ('CC', 'COCOS (KEELING) ISLANDS'),
    ('CO', 'COLOMBIA'),
    ('KM', 'COMOROS'),
    ('CG', 'CONGO'),
    ('CD', 'CONGO, THE DEMOCRATIC REPUBLIC OF'),
    ('CK', 'COOK ISLANDS'),
    ('CR', 'COSTA RICA'),
    ('CI', "CÃ”TE D'IVOIRE"),
    ('HR', 'CROATIA'),
    ('CU', 'CUBA'),
    ('CY', 'CYPRUS'),
    ('CZ', 'CZECH REPUBLIC'),
    ('DK', 'DENMARK'),
    ('DJ', 'DJIBOUTI'),
    ('DM', 'DOMINICA'),
    ('DO', 'DOMINICAN REPUBLIC'),
    ('EC', 'ECUADOR'),
    ('EG', 'EGYPT'),
    ('SV', 'EL SALVADOR'),
    ('GQ', 'EQUATORIAL GUINEA'),
    ('ER', 'ERITREA'),
    ('EE', 'ESTONIA'),
    ('ET', 'ETHIOPIA'),
    ('FK', 'FALKLAND ISLANDS (MALVINAS)'),
    ('FO', 'FAROE ISLANDS'),
    ('FJ', 'FIJI'),
    ('FI', 'FINLAND'),
    ('FR', 'FRANCE'),
    ('GF', 'FRENCH GUIANA'),
    ('PF', 'FRENCH POLYNESIA'),
    ('TF', 'FRENCH SOUTHERN TERRITORIES'),
    ('GA', 'GABON'),
    ('GM', 'GAMBIA'),
    ('GE', 'GEORGIA'),
    ('DE', 'GERMANY'),
    ('GH', 'GHANA'),
    ('GI', 'GIBRALTAR'),
    ('GR', 'GREECE'),
    ('GL', 'GREENLAND'),
    ('GD', 'GRENADA'),
    ('GP', 'GUADELOUPE'),
    ('GU', 'GUAM'),
    ('GT', 'GUATEMALA'),
    ('GN', 'GUINEA'),
    ('GW', 'GUINEA'),
    ('GY', 'GUYANA'),
    ('HT', 'HAITI'),
    ('HM', 'HEARD ISLAND AND MCDONALD ISLANDS'),
    ('HN', 'HONDURAS'),
    ('HK', 'HONG KONG'),
    ('HU', 'HUNGARY'),
    ('IS', 'ICELAND'),
    ('IN', 'INDIA'),
    ('ID', 'INDONESIA'),
    ('IR', 'IRAN, ISLAMIC REPUBLIC OF'),
    ('IQ', 'IRAQ'),
    ('IE', 'IRELAND'),
    ('IL', 'ISRAEL'),
    ('IT', 'ITALY'),
    ('JM', 'JAMAICA'),
    ('JP', 'JAPAN'),
    ('JO', 'JORDAN'),
    ('KZ', 'KAZAKHSTAN'),
    ('KE', 'KENYA'),
    ('KI', 'KIRIBATI'),
    ('KP', "KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF"),
    ('KR', 'KOREA, REPUBLIC OF'),
    ('KW', 'KUWAIT'),
    ('KG', 'KYRGYZSTAN'),
    ('LA', "LAO PEOPLE'S DEMOCRATIC REPUBLIC"),
    ('LV', 'LATVIA'),
    ('LB', 'LEBANON'),
    ('LS', 'LESOTHO'),
    ('LR', 'LIBERIA'),
    ('LY', 'LIBYAN ARAB JAMAHIRIYA'),
    ('LI', 'LIECHTENSTEIN'),
    ('LT', 'LITHUANIA'),
    ('LU', 'LUXEMBOURG'),
    ('MO', 'MACAO'),
    ('MK', 'MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF'),
    ('MG', 'MADAGASCAR'),
    ('MW', 'MALAWI'),
    ('MY', 'MALAYSIA'),
    ('MV', 'MALDIVES'),
    ('ML', 'MALI'),
    ('MT', 'MALTA'),
    ('MH', 'MARSHALL ISLANDS'),
    ('MQ', 'MARTINIQUE'),
    ('MR', 'MAURITANIA'),
    ('MU', 'MAURITIUS'),
    ('YT', 'MAYOTTE'),
    ('MX', 'MEXICO'),
    ('FM', 'MICRONESIA, FEDERATED STATES OF'),
    ('MD', 'MOLDOVA, REPUBLIC OF'),
    ('MC', 'MONACO'),
    ('MN', 'MONGOLIA'),
    ('MS', 'MONTSERRAT'),
    ('MA', 'MOROCCO'),
    ('MZ', 'MOZAMBIQUE'),
    ('MM', 'MYANMAR'),
    ('NA', 'NAMIBIA'),
    ('NR', 'NAURU'),
    ('NP', 'NEPAL'),
    ('NL', 'NETHERLANDS'),
    ('AN', 'NETHERLANDS ANTILLES'),
    ('NC', 'NEW CALEDONIA'),
    ('NZ', 'NEW ZEALAND'),
    ('NI', 'NICARAGUA'),
    ('NE', 'NIGER'),
    ('NG', 'NIGERIA'),
    ('NU', 'NIUE'),
    ('NF', 'NORFOLK ISLAND'),
    ('MP', 'NORTHERN MARIANA ISLANDS'),
    ('NO', 'NORWAY'),
    ('OM', 'OMAN'),
    ('PK', 'PAKISTAN'),
    ('PW', 'PALAU'),
    ('PS', 'PALESTINIAN TERRITORY, OCCUPIED'),
    ('PA', 'PANAMA'),
    ('PG', 'PAPUA NEW GUINEA'),
    ('PY', 'PARAGUAY'),
    ('PE', 'PERU'),
    ('PH', 'PHILIPPINES'),
    ('PN', 'PITCAIRN'),
    ('PL', 'POLAND'),
    ('PT', 'PORTUGAL'),
    ('PR', 'PUERTO RICO'),
    ('QA', 'QATAR'),
    ('RE', 'RÃ‰UNION'),
    ('RO', 'ROMANIA'),
    ('RU', 'RUSSIAN FEDERATION'),
    ('RW', 'RWANDA'),
    ('SH', 'SAINT HELENA'),
    ('KN', 'SAINT KITTS AND NEVIS'),
    ('LC', 'SAINT LUCIA'),
    ('PM', 'SAINT PIERRE AND MIQUELON'),
    ('VC', 'SAINT VINCENT AND THE GRENADINES'),
    ('WS', 'SAMOA'),
    ('SM', 'SAN MARINO'),
    ('ST', 'SAO TOME AND PRINCIPE'),
    ('SA', 'SAUDI ARABIA'),
    ('SN', 'SENEGAL'),
    ('CS', 'SERBIA AND MONTENEGRO'),
    ('SC', 'SEYCHELLES'),
    ('SL', 'SIERRA LEONE'),
    ('SG', 'SINGAPORE'),
    ('SK', 'SLOVAKIA'),
    ('SI', 'SLOVENIA'),
    ('SB', 'SOLOMON ISLANDS'),
    ('SO', 'SOMALIA'),
    ('ZA', 'SOUTH AFRICA'),
    ('GS', 'SOUTH GEORGIA AND SOUTH SANDWICH ISLANDS'),
    ('ES', 'SPAIN'),
    ('LK', 'SRI LANKA'),
    ('SD', 'SUDAN'),
    ('SR', 'SURINAME'),
    ('SJ', 'SVALBARD AND JAN MAYEN'),
    ('SZ', 'SWAZILAND'),
    ('SE', 'SWEDEN'),
    ('CH', 'SWITZERLAND'),
    ('SY', 'SYRIAN ARAB REPUBLIC'),
    ('TW', 'TAIWAN, PROVINCE OF CHINA'),
    ('TJ', 'TAJIKISTAN'),
    ('TZ', 'TANZANIA, UNITED REPUBLIC OF'),
    ('TH', 'THAILAND'),
    ('TL', 'TIMOR'),
    ('TG', 'TOGO'),
    ('TK', 'TOKELAU'),
    ('TO', 'TONGA'),
    ('TT', 'TRINIDAD AND TOBAGO'),
    ('TN', 'TUNISIA'),
    ('TR', 'TURKEY'),
    ('TM', 'TURKMENISTAN'),
    ('TC', 'TURKS AND CAICOS ISLANDS'),
    ('TV', 'TUVALU'),
    ('UG', 'UGANDA'),
    ('UA', 'UKRAINE'),
    ('AE', 'UNITED ARAB EMIRATES'),
    ('GB', 'UNITED KINGDOM'),
    ('US', 'UNITED STATES'),
    ('UM', 'UNITED STATES MINOR OUTLYING ISLANDS'),
    ('UY', 'URUGUAY'),
    ('UZ', 'UZBEKISTAN'),
    ('VU', 'VANUATU'),
    ('VN', 'VIET NAM'),
    ('VG', 'VIRGIN ISLANDS, BRITISH'),
    ('VI', 'VIRGIN ISLANDS, U.S.'),
    ('WF', 'WALLIS AND FUTUNA'),
    ('EH', 'WESTERN SAHARA'),
    ('YE', 'YEMEN'),
    ('ZW', 'ZIMBABWE')
)

In [ ]:
list_of_countries = []
for i in countries:
    list_of_countries.append(i[1].lower())

In [ ]:
users_df = users_df[users_df['Country'].isin(list_of_countries)]
users_df

,User-ID,Age,City,State,Country
3,4,17,porto,v.n.gaia,portugal
4,5,34,farnborough,hants,united kingdom
7,8,34,timmins,ontario,canada
9,10,26,albacete,wisconsin,spain
10,11,14,melbourne,victoria,australia
...,...,...,...,...,...
278851,278852,32,brisbane,queensland,australia
278852,278853,17,stranraer,n/a,united kingdom
278854,278855,50,tacoma,washington,united kingdom
278855,278856,34,brampton,ontario,canada


In [ ]:
list_of_states = users_df['State'].str.extract(r'(^[a-z]{4}[a-z]*|^[a-z]{1}[a-z]*\w[a-z]{4})')[0].unique()
users_df = users_df[users_df['State'].isin(list_of_states)]
list_of_city = users_df['City'].str.extract(r'(^[a-z]{4}[a-z]*|^[a-z]{1}[a-z]*\w[a-z]{4})')[0].unique()
users_df = users_df[users_df['City'].isin(list_of_city)]

In [ ]:
print("Cantidad de paises únicos: " , users_df['Country'].nunique())
print("Cantidad de estados únicos: " ,users_df['State'].nunique())
print("Cantidad de ciudades únicas: " , users_df['City'].nunique())

Cantidad de paises únicos:  154
Cantidad de estados únicos:  2193
Cantidad de ciudades únicas:  8457


## Limpieza data set books

In [ ]:
books_df = books_df.drop(['Image-URL-S','Image-URL-M','Image-URL-L'], axis=1)
year_of_publication = books_df['Year-Of-Publication'].str.extract(r'(\d{4})')[0]
books_df = books_df[books_df['Year-Of-Publication'].isin(year_of_publication)]
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(int)
books_df.drop_duplicates(subset='ISBN', inplace=True)
books_df.head()

<ipython-input-69-3575220846>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


## Limpieza data set ratings

In [ ]:
ratings_df = ratings_df.dropna(subset=['User-ID', 'ISBN', 'Book-Rating'])
ratings_df = ratings_df[ratings_df['Book-Rating'].between(0, 10)]
ratings_df.drop_duplicates(inplace=True)
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
ratings_df['Book-Rating'] = pd.to_numeric(ratings_df['Book-Rating'], errors='coerce').astype('int')
ratings_graph_params = ratings_df.groupby('Book-Rating').size().sort_index()
fig = go.Figure(
    data=[
        go.Bar(
            x=ratings_graph_params.index,
            y=ratings_graph_params.values
        )
    ]
)
fig.update_layout(
    title='Distribución por calificación',
    xaxis_title='Calificación',
    yaxis_title='Frecuencia'
)
fig.show()

In [ ]:
# 🧪 Vista previa de los datos limpios
print("Books:")
print(books_df.head())
print("\nRatings:")
print(ratings_df.head())
print("\nUsers:")
print(users_df.head())

Books:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author  Year-Of-Publication                   Publisher  
0    Mark P. O. Morford                 2002     Oxford University Press  
1  Richard Bruce Wright                 2001       HarperFlamingo Canada  
2          Carlo D'Este                 1991             HarperPerennial  
3      Gina Bari Kolata                 1999        Farrar Straus Giroux  
4       E. J. W. Barber                 1999  W. W. Norton &amp; Company  

Ratings:
   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446

# Preparar dataset final para clustering con K-Means

In [ ]:
# Fusionamos ratings con usuarios y libros para tener un dataset enriquecido
# Use the correct variable names after cleaning
users_df['User-ID'] = users_df['User-ID'].astype(int)

dataset_final = ratings_df.merge(books_df,on = 'ISBN').merge(users_df,how='inner',on='User-ID')

# Verificación de dataset tras el merge
dataset_final = dataset_final.drop_duplicates(['User-ID','ISBN'],keep='first')
dataset_final.head()


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,City,State,Country
0,276774,0099543818,0,"East, West",Salman Rushdie,1995,Vintage Books,28,bern,bern,switzerland
1,276774,3404147723,0,Im Schatten der Lilie. Die Erinnerungen der El...,Patrice Leavold,2002,LÃ?Â¼bbe,28,bern,bern,switzerland
2,276774,3423111321,0,Die Teufelin. Roman.,Fay Weldon,1989,Dtv,28,bern,bern,switzerland
3,276774,3442136644,9,Endlich Nichtraucher.,Allen Carr,1992,Goldmann,28,bern,bern,switzerland
4,276774,3492232000,0,Ein Liebhaber zuviel ist noch zu wenig.,Gaby Hauptmann,2000,Piper,28,bern,bern,switzerland


In [ ]:
dataset_final = dataset_final[['Book-Rating','Book-Author','Year-Of-Publication','Publisher','Age','City','State','Country']].copy()
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133180 entries, 0 to 133179
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Book-Rating          133180 non-null  int64 
 1   Book-Author          133179 non-null  object
 2   Year-Of-Publication  133180 non-null  int64 
 3   Publisher            133180 non-null  object
 4   Age                  133180 non-null  int64 
 5   City                 133180 non-null  object
 6   State                133180 non-null  object
 7   Country              133180 non-null  object
dtypes: int64(3), object(5)
memory usage: 8.1+ MB


## Codificar datos Categóricos

In [ ]:
encoder = LabelEncoder()
for column in dataset_final.columns:
    if dataset_final[column].dtype == object:
        dataset_final[column] = encoder.fit_transform(dataset_final[column])

In [ ]:
dataset_final

,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Age,City,State,Country
0,0,27899,1995,6299,28,261,81,79
1,0,23956,2002,3642,28,261,81,79
2,0,9475,1989,1700,28,261,81,79
3,9,1011,1992,2424,28,261,81,79
4,0,10256,2000,4680,28,261,81,79
...,...,...,...,...,...,...,...,...
133175,0,21335,1997,3030,30,2546,496,16
133176,0,15063,2000,1330,30,2546,496,16
133177,5,15517,1999,6006,34,1831,264,87
133178,5,21169,2000,6006,34,1831,264,87


In [ ]:
dataset_final.to_csv('DataSetFinal.csv', index=False)